# Building Fast Queries on a CSV

In this project, we will look to build a way to answer a few different business questions about the inventory for an online laptop store.

## The Dataset

We will be using the `laptops.csv` file as our inventory. This CSV file was adapted from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices), which contains information regarding characteristics and prices for 1300 laptops. In our version of the dataset, the IDs have been changed and the prices have been converted to integers.

A description of what each column contains can be found in the link above. Please note that the ID column has been omitted.

Let's start explore the data and print the first few rows before we start implementing a way to represent this data as the store inventory.





In [1]:
# Read in the CSV file and print the first few rows.
import csv

with open('laptops.csv', mode='r') as file:
    rows = list(csv.reader(file))
    header = rows[0]
    rows = rows[1:]

print(header)
print(rows[:5])


['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB 

## Inventory Class

One way to approach our project is to create a class that represents our inventory. We can then create methods within the class that will implement the queries that we want to answer. We can also preprocess the data to make those queries run faster.

Here are a few examples of queries that we will want to answer:


* Given a laptop id, find the corresponding data.
* Given an amount of money, find whether there are two laptops whose total price is that given amount.
* Identify all laptops whose price falls within a given budget.

Let's start by implementing the constructor. It will take the name of the CSV file as argument and then read the rows contained within it.

In [2]:
# Create an Inventory class and define the constructor
class Inventory():
    
    def __init__(self, csv_filename):
        with open('laptops.csv', mode='r') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                
test = Inventory('laptops.csv')
print(test.header)
print(len(test.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


We can see that our `Inventory` class and constructor work as intended. We can also see that our dataset countains 1303 laptops.

## Finding a Laptop From the ID

The first improvement to our `Inventory` class that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

For this, we will write a function named `get_laptop_from_id()`. This function will take as argument the identifier of the laptop and return the full row of the laptop with that ID.

In [3]:
# Write a method that looks up a laptop using the ID.
class Inventory():
    
    def __init__(self, csv_filename):
        with open('laptops.csv', mode='r') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None


test = Inventory('laptops.csv')
print(test.get_laptop_from_id('3362737'))
print(test.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Improving ID Lookups

The algorithm above requires us to look at every single row to find the one we are looking for (or decide that such a row does not exist). This Algorithm has a time complexity of O(R), where R is the number of rows.

Lets improve on this algorithm by preprocessing the data into a dictionary. This will let us look up the IDs in constant time and return the row as the key's value.
As a trade off, this will increase the algorithm's space complexity.


In [4]:
# Improve the get_laptop_from_id() method by preprocessing the data into a dictionary.
class Inventory():
    
    def __init__(self, csv_filename):
        with open('laptops.csv', mode='r') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    # Improved algorithm that uses a dictionary to allow constant-time lookups.
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
test = Inventory('laptops.csv')
print(test.get_laptop_from_id_fast('3362737'))
print(test.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Comparing the Performance

While the orginial `get_laptop_from_id()` method had a time complexity of O(R), where R is the number of rows, the new implementation has a time complexity of O(1).
It does so by using more memory to store the `self.id_to_row` dictionary and using a bit more time to create an instance.

Let's experiment to compare the performance of the two methods.

In [5]:
import time
import random

# Generate 10,000 random laptop IDs.
ids = str([random.randint(1000000, 9999999) for _ in range(10000)])

test = Inventory('laptops.csv')

# Calculate the total runtime to lookup all IDs using the get_laptop_from_id() method.
total_time_no_dict = 0
for id in ids:
    start = time.time()
    test.get_laptop_from_id(id)
    end = time.time()
    total_time_no_dict += end - start
    
# Calculate the total runtime to lookup all IDs using the get_laptop_from_id_fast() method.
total_time_dict = 0
for id in ids:
    start = time.time()
    test.get_laptop_from_id_fast(id)
    end = time.time()
    total_time_dict += end - start
    
print(total_time_no_dict)
print(total_time_dict)

8.893755435943604
0.04534649848937988


We can see here that using the `get_laptop_from_id_fast` method makes a huge difference in runtime when we need to run a lot of queries.

## Two Laptop Promotion

Sometimes, the store offers a promotion where we give a gift card. A customer can use to gift card to buy up to two laptops and any leftover credit is forfeited.

In order to be fair to our customers, we must ensure that it is always possible to use all of the credit whenever the promotion is running. This means that there needs to be a single laptop that costs the full amount or two laptops that total the full amount when paired.

Let's write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.


In [6]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open('laptops.csv', mode='r') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    # A method that checks whether a single laptop or a combination of two laptops totals a certain cost.
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row_1 in self.rows:
            for row_2 in self.rows:
                if row_1[-1] + row_2[-1] == dollars:
                    return True
        return False

test = Inventory('laptops.csv')
print(test.check_promotion_dollars(1000))
print(test.check_promotion_dollars(442))

True
False


## Optimising Laptop Promotion

The alogorithm above has a time complexity of O(R^2), where R is the number of rows.
As before, we can optimise the algorithm by preprocessing the data. Here we can use a set as we only need the laptop prices. This will allow us to peform our queries in constant time.

In [7]:
# Preprocess the data as a set.
class Inventory():
    
    def __init__(self, csv_filename):
        with open('laptops.csv', mode='r') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
            self.prices = set()
            for row in self.rows:
                self.prices.add(row[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row_1 in self.rows:
            for row_2 in self.rows:
                if row_1[-1] + row_2[-1] == dollars:
                    return True
        return False
    
    # An improved method that allows us to lookup in constant time using a set.
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for row in self.rows:
            if dollars - row[-1] in self.prices:
                return True
        return False
    
test = Inventory('laptops.csv')
print(test.check_promotion_dollars_fast(1000))
print(test.check_promotion_dollars_fast(442))

True
False


## Comparing Promotion Funtions

Let's compare the performance of the last two funtions that we wrote.

In [8]:
# Generate a list of random target prices
prices = [random.randint(100, 5000) for _ in range(100)]

test = Inventory('laptops.csv')

# Calculate the total runtime to query all target prices using the original check_promotion_dollars() funtion.
total_time_no_set = 0
for price in prices:
    start = time.time()
    test.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end - start
    
# Calculate the total runtime to query all target prices using the improved check_promotion_dollars_fast() funtion.
total_time_set = 0
for price in prices:
    start = time.time()
    test.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end - start
    
print(total_time_no_set)
print(total_time_set)

2.0857114791870117
0.0015993118286132812


As before, we can see that preprocessing data into a more appropriate datatype for lookups can have a huge impact on runtimes when a large number of queries are needed.

## Finding Laptops Within a Budget

We will now leverage a binary search algorithm to write a method that efficiently queries the dataset to find all laptops that fall within a given budget.

In [10]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open('laptops.csv', mode='r') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
            self.prices = set()
            for row in self.rows:
                self.prices.add(row[-1])
            # Sort the rows by laptop price.
            self.rows_by_price = sorted(self.rows, key=lambda row_price: self.rows[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row_1 in self.rows:
            for row_2 in self.rows:
                if row_1[-1] + row_2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for row in self.rows:
            if dollars - row[-1] in self.prices:
                return True
        return False
    
    # A method to find the index of the first laptop that is more expensive that a given amount.
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        
        while range_start < range_end:
            range_middle = (range_start + range_end) // 2
            row_price = self.rows_by_price[range_middle][-1]
            if row_price <= target_price:
                range_start = range_middle + 1
            else:
                range_end = range_middle
        row_price = self.rows_by_price[range_start][-1]
        if row_price > target_price:
            return range_start
        return -1

        
test = Inventory('laptops.csv')
print(test.find_first_laptop_more_expensive(1000))
print(test.find_first_laptop_more_expensive(10000))

                    
                
                

917
-1
